# RNN Test: Recetas de cocina

Basado en: https://www.kdnuggets.com/2020/07/generating-cooking-recipes-using-tensorflow.html

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import json

import platform
import time
import pathlib
import os
import pickle

In [2]:
# Se esconde la GPU porque al parecer hay problemas con cuDNN y LSTM
# https://stackoverflow.com/questions/37660312/how-to-run-tensorflow-on-cpu
# https://github.com/mozilla/DeepSpeech/issues/3088
# 
# tf.config.set_visible_devices([], 'GPU')
# os.environ['CUDA_VISIBLE_DEVICES'] = ''

# Cargar los datos

In [3]:
dataset_targeted = tf.data.experimental.load('Dataset_Train', compression='GZIP')
with open('Tokenizer.pkl', 'rb') as file:
    tokenizer = pickle.load(file)

In [4]:
VOCABULARY_SIZE = len(tokenizer.word_counts) + 1

# Separar la base de datos en batches

In [5]:
# Batch size.
BATCH_SIZE = 64

# Buffer size to shuffle the dataset (TF data is designed to work
# with possibly infinite sequences, so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in
# which it shuffles elements).
SHUFFLE_BUFFER_SIZE = 1000

dataset_train = dataset_targeted.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).repeat()

print(dataset_train)

<RepeatDataset shapes: ((64, 2000), (64, 2000)), types: (tf.int32, tf.int32)>


# Construir el modelo

In [6]:
tmp_vocab_size = 10
tmp_embedding_size = 5
tmp_input_length = 8
tmp_batch_size = 2

tmp_model = tf.keras.models.Sequential()
tmp_model.add(tf.keras.layers.Embedding(
  input_dim=tmp_vocab_size,
  output_dim=tmp_embedding_size,
  input_length=tmp_input_length
))
# The model will take as input an integer matrix of size (batch, input_length).
# The largest integer (i.e. word index) in the input should be no larger than 9 (tmp_vocab_size).
# Now model.output_shape == (None, 10, 64), where None is the batch dimension.
tmp_input_array = np.random.randint(
  low=0,
  high=tmp_vocab_size,
  size=(tmp_batch_size, tmp_input_length)
)
tmp_model.compile('rmsprop', 'mse')
tmp_output_array = tmp_model.predict(tmp_input_array)

print('tmp_input_array shape:', tmp_input_array.shape)
print('tmp_input_array:')
print(tmp_input_array)
print()
print('tmp_output_array shape:', tmp_output_array.shape)
print('tmp_output_array:')
print(tmp_output_array)

tmp_input_array shape: (2, 8)
tmp_input_array:
[[9 3 4 9 8 7 7 0]
 [7 9 5 1 2 4 9 2]]

tmp_output_array shape: (2, 8, 5)
tmp_output_array:
[[[ 0.00891067 -0.02277729 -0.03066033  0.0083333  -0.00823761]
  [ 0.03773025  0.0295834  -0.02404933  0.043337    0.01378472]
  [-0.00171058 -0.03350209 -0.00853221  0.00702941 -0.004572  ]
  [ 0.00891067 -0.02277729 -0.03066033  0.0083333  -0.00823761]
  [ 0.00666205  0.0015915  -0.02814863 -0.03730105  0.0387584 ]
  [ 0.04477696  0.01274076 -0.00994901 -0.0189967  -0.00352753]
  [ 0.04477696  0.01274076 -0.00994901 -0.0189967  -0.00352753]
  [-0.01440569  0.01686419 -0.02905114 -0.04770547  0.02911556]]

 [[ 0.04477696  0.01274076 -0.00994901 -0.0189967  -0.00352753]
  [ 0.00891067 -0.02277729 -0.03066033  0.0083333  -0.00823761]
  [ 0.00074028 -0.03683481 -0.0330416   0.00641272 -0.01775007]
  [-0.01117425 -0.00350052 -0.03661614  0.01532303  0.00987228]
  [-0.03477343 -0.01002655  0.00635991 -0.01404545  0.00954934]
  [-0.00171058 -0.03350209 

# Modelo LSTM

In [7]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        batch_input_shape=[batch_size, None]
    ))

    model.add(tf.keras.layers.GRU(
        units=rnn_units,
        return_sequences=True,
        stateful=True,
        recurrent_initializer=tf.keras.initializers.GlorotNormal()
    ))

    model.add(tf.keras.layers.Dense(vocab_size))
    
    return model

model = build_model(
  vocab_size=VOCABULARY_SIZE,
  embedding_dim=256,
  rnn_units=1024,
  batch_size=BATCH_SIZE
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           45056     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 176)           180400    
Total params: 4,163,760
Trainable params: 4,163,760
Non-trainable params: 0
_________________________________________________________________


# Pruebas del modelo

In [8]:
for input_example_batch, target_example_batch in dataset_train.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

InternalError: Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 3, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 256, 1024, 1, 2000, 64, 0]  [Op:CudnnRNN]

In [ ]:
print('Prediction for the 1st letter of the batch 1st sequense:')
print(example_batch_predictions[0, 0])

# Pruebas con `tf.random.categorical`

In [ ]:
# logits is 2-D Tensor with shape [batch_size, num_classes].
# Each slice [i, :] represents the unnormalized log-probabilities for all classes.
# In the example below we say that the probability for class "0"
# (element with index 0) is low but the probability for class "2" is much higher.
tmp_logits = [
  [-0.95, 0, 0.95],
];

# Let's generate 5 samples. Each sample is a class index. Class probabilities 
# are being taken into account (we expect to see more samples of class "2").
tmp_samples = tf.random.categorical(
    logits=tmp_logits,
    num_samples=5
)

print(tmp_samples)

In [ ]:
sampled_indices = tf.random.categorical(
    logits=example_batch_predictions[0],
    num_samples=1
)

sampled_indices = tf.squeeze(
    input=sampled_indices,
    axis=-1
).numpy()

sampled_indices.shape

In [12]:
print('Input:\n', repr(''.join(tokenizer.sequences_to_texts([input_example_batch[0].numpy()[:50]]))))
print()
print('Next char prediction:\n', repr(''.join(tokenizer.sequences_to_texts([sampled_indices[:50]]))))

Input:
 '📗   O l d   F a s h i o n e d   O n i o n   R i n g s \n \n 🥕 \n \n •   1   l a r g e   o n i o n ,   c'

Next char prediction:
 '9 r ? ` L 5 n ‟ – P ’ ] F 🥕 » û w B D t N ⅓ \x95 \n O t 4 » V ⅞ J ê É € ä S ⅓ a ⁄ > m ó a ư j d € W'


# Entrenando el modelo

In [13]:
# An objective function.
# The function is any callable with the signature scalar_loss = fn(y_true, y_pred).
def loss(labels, logits):
    entropy = tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )
    
    return entropy

example_batch_loss = loss(target_example_batch, example_batch_predictions)

print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss.shape:      ", example_batch_loss.shape)
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 2000, 176)  # (batch_size, sequence_length, vocab_size)
scalar_loss.shape:       (64, 2000)
scalar_loss:       5.170606


In [14]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=adam_optimizer,
    loss=loss
)

In [15]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=5,
    monitor='loss',
    restore_best_weights=True,
    verbose=1
)

In [16]:
# Create a checkpoints directory.
checkpoint_dir = 'tmp/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [17]:
EPOCHS = 500
INITIAL_EPOCH = 1
STEPS_PER_EPOCH = 1500

print('EPOCHS:          ', EPOCHS)
print('INITIAL_EPOCH:   ', INITIAL_EPOCH)
print('STEPS_PER_EPOCH: ', STEPS_PER_EPOCH)

EPOCHS:           500
INITIAL_EPOCH:    1
STEPS_PER_EPOCH:  1500


In [ ]:
history = model.fit(x=dataset_train,
    epochs=EPOCHS,steps_per_epoch=STEPS_PER_EPOCH,
    initial_epoch=INITIAL_EPOCH,
    callbacks=[checkpoint_callback, early_stopping_callback],
    verbose=1)

# Saving the trained model to file (to be able to re-use it later).
model_name = 'recipe_generation_rnn_raw.h5'
model.save(model_name, save_format='h5')

Epoch 2/500
   2/1500 [..............................] - ETA: 29:40:43 - loss: 4.9623

In [9]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1789427665321178190
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1996200347
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3093676066046094884
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Default GPU Device: /device:GPU:0
